# Data Preparation

In [800]:
import geopandas as gpd
from osgeo import ogr

import gdal
import numpy as np
from pathlib import Path
import ogr
import pandas as pd
import rasterio
import gdal
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

- A) Original Rio-Segura file (Check a structure of file)

In [801]:
cro_org = Path(f'../1_Documents/SR AH 2020-21 desagregado leñosos/Cultivos_regados_AH2021_v6.shp')
cro_org = gpd.read_file(cro_org)
cro_org = pd.DataFrame(cro_org)
cro_org.shape

(277695, 22)

In [804]:
cro_org.head(5)
cro_org.iloc[:,:18].head(5)

,Superficie,C20_M10,C20_M11,C20_M12,C21_M01,C21_M02,C21_M03,C21_M04,C21_M05,C21_M06,C21_M07,C21_M08,C21_M09,HI2021,HP2021,HV2021,HO20,C_AH2021
0,462,None,None,None,None,None,None,None,H0,H,H,None,None,None,None,None,None,HU
1,459,None,None,None,None,None,None,None,H0,H,H,None,None,None,None,None,None,HU
2,298,None,None,None,None,None,None,None,H0,H,H,None,None,None,None,None,None,HU
3,537,None,None,None,None,None,None,None,H0,H,H,None,None,None,None,None,None,HU
4,151,None,None,None,None,None,None,None,H0,H,H,None,None,None,None,None,None,HU


In [805]:
cro_org.columns

Index(['Superficie', 'C20_M10', 'C20_M11', 'C20_M12', 'C21_M01', 'C21_M02',
       'C21_M03', 'C21_M04', 'C21_M05', 'C21_M06', 'C21_M07', 'C21_M08',
       'C21_M09', 'HI2021', 'HP2021', 'HV2021', 'HO20', 'C_AH2021',
       'MESES_RIEG', 'COSECHAS', 'Cat_L_SP', 'geometry'],
      dtype='object')

- B) Original Rio-Segura file Translated (Check a structure)

In [737]:
crop_2021 =  gpd.read_file('Final/crop_2021.shp')
crop_2021.shape

(277695, 25)

In [731]:
crop_2021.head(5)

,Superficie,C20_M10,C20_M11,C20_M12,C21_M01,C21_M02,C21_M03,C21_M04,C21_M05,C21_M06,...,Herb_Summe,Herb_Autum,Seasons,TotalCropM,NumberHarv,TreeType,TreeType_T,Seasons_T,Training,geometry
0,462,None,None,None,None,None,None,None,H0,H,...,None,None,HU,3,1,None,None,Family gardens,H,"POLYGON ((537406.530500001 4216862.771999994, ..."
1,459,None,None,None,None,None,None,None,H0,H,...,None,None,HU,3,1,None,None,Family gardens,H,"POLYGON ((537406.530500001 4216862.771999994, ..."
2,298,None,None,None,None,None,None,None,H0,H,...,None,None,HU,3,1,None,None,Family gardens,H,"POLYGON ((537315.0019999996 4216616.410499991,..."
3,537,None,None,None,None,None,None,None,H0,H,...,None,None,HU,3,1,None,None,Family gardens,H,"POLYGON ((537317.1019999986 4216599.030499995,..."
4,151,None,None,None,None,None,None,None,H0,H,...,None,None,HU,3,1,None,None,Family gardens,H,"POLYGON ((537382.4605000004 4216870.851999988,..."


In [732]:
crop_2021.columns

Index(['Superficie', 'C20_M10', 'C20_M11', 'C20_M12', 'C21_M01', 'C21_M02',
       'C21_M03', 'C21_M04', 'C21_M05', 'C21_M06', 'C21_M07', 'C21_M08',
       'C21_M09', 'Herb_Winte', 'Herb_Sprin', 'Herb_Summe', 'Herb_Autum',
       'Seasons', 'TotalCropM', 'NumberHarv', 'TreeType', 'TreeType_T',
       'Seasons_T', 'Training', 'geometry'],
      dtype='object')

# Our Crop Classification Deliverables

In [817]:
perm =  gpd.read_file('crop-predictions.geojson')
herb =  gpd.read_file('herbaceous-probabilties.geojson')
print("Perm:", perm.shape, "Herb:",herb.shape)

Perm: (277199, 30) Herb: (277249, 33)


In [924]:
gdf = perm.merge(herb,on='xy',  how='left')
gdf = gdf.drop_duplicates(subset='xy',keep='first')
gdf.prediction.dropna()
gdf = merge_unique[['2101','2102','2103','2104','2105','2106','2107','2108','2109','2010','2011','2012','prediction', 'geometry_y','Superficie']]
print("GDF merged:", gdf.shape)

# Raname Columns and clsas values
gdf['Class'] = gdf['prediction'].replace({0:'Herbaceous',   1:'Low-density',
                                          2:'High-density', 3:'Vineyard',
                                          4:'Greenhouse',   5:'Family Gardens'})

gdf['Class_ES'] = gdf['Class'].replace({'Herbaceous':'H', 'Low-density':'LBC',
                                        'High-density': 'LAC',  'Vineyard':'VP',
                                        'Greenhouse':'INV',     'Family Gardens':'HU'})

gdf = gdf.rename(columns={'2101':'C21_M01','2102':'C21_M02','2103':'C21_M03','2104':'C21_M04',
                          '2105':'C21_M05','2106':'C21_M06','2107':'C21_M07','2108':'C21_M08',
                          '2109':'C21_M09','2010':'C20_M10','2011':'C20_M11','2012':'C20_M12'})

GDF merged: (277031, 15)


/home/jovyan/.conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/jovyan/.conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [925]:
for ind, column_name in enumerate(gdf.columns[:12]):
    gdf[column_name] = np.where((gdf.prediction == 0) & (gdf[column_name] >= 0.4), 1, gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction == 5) & (gdf[column_name] >= 0.4), 1, gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction == 0) & (gdf[column_name] <= 0.4), np.nan, gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction == 5) & (gdf[column_name] <= 0.4), np.nan, gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction != 0) & (gdf.Class_ES == 'LAC'), 'LAC', gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction != 0) & (gdf.Class_ES == 'LBC'), 'LBC', gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction == 4), np.nan, gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction == 5) & (gdf[column_name]== '1.0'), "H", gdf[column_name])
    gdf[column_name] = np.where((gdf.prediction == 0) & (gdf[column_name] == '1.0'), 'H', gdf[column_name])
    

In [926]:
gdf = gdf.rename(columns={'geometry_y':'geometry'})
gdf = gdf.drop(columns=['prediction','Class'])
gdf = gpd.GeoDataFrame(gdf, crs="EPSG:4326", geometry='geometry')

In [927]:
gdf["geometry"] = [MultiPolygon([feature]) if isinstance(feature, Polygon) \
    else feature for feature in gdf["geometry"]]

In [928]:
gdf.shape

(277031, 15)

In [933]:
gdf

,C21_M01,C21_M02,C21_M03,C21_M04,C21_M05,C21_M06,C21_M07,C21_M08,C21_M09,C20_M10,C20_M11,C20_M12,geometry,Superficie,Class_ES
0,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,(POLYGON ((-1.123112632649344 37.9534337461708...,3134,LAC
1,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,(POLYGON ((-0.8278106218991994 37.973399717851...,27430,LAC
2,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,(POLYGON ((-1.075998440878904 37.9576870404691...,586,LAC
3,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,(POLYGON ((-0.9260390227860397 37.886147792730...,18192,LBC
4,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,(POLYGON ((-0.8789629429613586 37.846351574096...,311,LBC
5,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,(POLYGON ((-1.03057539757508 37.66291525876795...,2283,LAC
6,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,(POLYGON ((-1.061221777404064 37.9616536969241...,3235,LAC
7,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,(POLYGON ((-1.050327748891738 37.9638220112831...,275,LAC
8,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,LBC,(POLYGON ((-1.115151996652562 38.0607409874936...,17485,LBC
9,H,H,H,H,H,H,nan,nan,nan,H,H,H,(POLYGON ((-0.9850491729557022 37.789328581361...,6267,H


In [790]:
gdf.to_file('CropClassification2021__May2022.gpkg', driver='GPKG')

(277031, 16)